# 1. Chicken farm schema

Created a simple schema in a google sheet [here](https://docs.google.com/spreadsheets/d/1Ir5d-4LADww_FJOt9VXVKb_TuFAyq2BsYR2F7R3FYl0/edit#gid=0)

# 2a. Generating data

In [78]:
import pandas as pd
import random
import duckdb 
from faker import Faker 

fake = Faker()

# number of chickens/eggs to generate
n = 1000

def generate_ids(num_chickens):
    identifiers = set() #ensure uniqueness in an empty set
    while len(identifiers) < num_chickens:
        identifier = random.randint(10000000, 99999999)
        identifiers.add(identifier)
    return list(identifiers)

chickens = generate_ids(n)
chicken_df = pd.DataFrame(chickens, columns=['egg_id'])

In [79]:
# because we need to create lineage, i'm creating 'generations', where generation 3 is the most recent set of chickens
# gen 3's parents are gen 2, and gen 2's parents are gen 1
# assuming hens give birth to a lot of chickens, so setting a 10-30-60 ratio across gens 1 thru 3

num_gen1 = int(n * 0.1)
num_gen2 = int(n * 0.3)
num_gen3 = n - (num_gen1 + num_gen2)

generation_list = [1] * num_gen1 + [2] * num_gen2 + [3] * num_gen3
chicken_df['generation'] = generation_list

chicken_df

,egg_id,generation
0,25804801,1
1,37570562,1
2,46751747,1
3,29038594,1
4,43585541,1
...,...,...
995,72511480,3
996,17602553,3
997,95074299,3
998,80680956,3


In [80]:
# function to assign rooster or hen to each chicken
def assign_gender_with_split(chickens, gender_split, gender1, gender2):
    half = len(chickens) // 2
    chickens.loc[chickens.index[:half], 'gender'] = gender1
    chickens.loc[chickens.index[half:], 'gender'] = gender2
    return chickens

# ensure 50-50 split between gender within each generation
gen1_df = chicken_df[chicken_df['generation'] == 1].copy()
gen2_df = chicken_df[chicken_df['generation'] == 2].copy()
gen3_df = chicken_df[chicken_df['generation'] == 3].copy()

gen1_df = assign_gender_with_split(gen1_df, 0.5, 'Rooster', 'Hen')
gen2_df = assign_gender_with_split(gen2_df, 0.5, 'Rooster', 'Hen')
gen3_df = assign_gender_with_split(gen3_df, 0.5, 'Rooster', 'Hen')

chicken_df = pd.concat([gen1_df, gen2_df, gen3_df], ignore_index=True)
chicken_df

,egg_id,generation,gender
0,25804801,1,Rooster
1,37570562,1,Rooster
2,46751747,1,Rooster
3,29038594,1,Rooster
4,43585541,1,Rooster
...,...,...,...
995,72511480,3,Hen
996,17602553,3,Hen
997,95074299,3,Hen
998,80680956,3,Hen


In [81]:
# let's create a lookup table for the incubation hall

incubation_hall_location_ids = list(range(1, 31))
is_near_window = [True if i <= 10 else False for i in incubation_hall_location_ids]

data = {
    "incubation_hall_location_id": incubation_hall_location_ids,
    "is_near_window": is_near_window
}

incubation_hall = pd.DataFrame(data)


In [82]:
# technically this is making it possible for a mother hen to have multiple incubation locations, but i'm assuming that's fine
chicken_df['incubation_hall_location_id'] = [random.choice(incubation_hall_location_ids) if generation != 1 else None for generation in chicken_df['generation']]

# check to make sure that roughly 33% of chickens were incubated near a window
duckdb.query("""
with counts as (
    select incubation_hall_location_id, count(*) as count from chicken_df where generation != 1 group by 1 order by 1
)

, windowed_sums as (
select
    *,
    sum(count) over (order by incubation_hall_location_id rows between unbounded preceding and current row) as cumulative_count,
    sum(count) over (order by incubation_hall_location_id rows between unbounded preceding and unbounded following) as total_count
from counts
)

select
    *,
    cumulative_count::float / total_count::float as cumulative_percent
from windowed_sums
 """).to_df()


,incubation_hall_location_id,count,cumulative_count,total_count,cumulative_percent
0,1.0,26,26.0,900.0,0.028889
1,2.0,28,54.0,900.0,0.060000
2,3.0,21,75.0,900.0,0.083333
3,4.0,30,105.0,900.0,0.116667
4,5.0,28,133.0,900.0,0.147778
5,6.0,29,162.0,900.0,0.180000
6,7.0,38,200.0,900.0,0.222222
7,8.0,36,236.0,900.0,0.262222
8,9.0,34,270.0,900.0,0.300000
9,10.0,23,293.0,900.0,0.325556


In [83]:
# validate that the geneder split function works correctly
duckdb.query("select generation, gender, count(*) as count from chicken_df group by 1,2").to_df()

,generation,gender,count
0,1,Rooster,50
1,1,Hen,50
2,2,Rooster,150
3,2,Hen,150
4,3,Rooster,300
5,3,Hen,300


In [84]:
# using the faker library to generate full names for each chicken dependent on gender
chicken_df['name'] = [
    f"{fake.first_name_male()} {fake.last_name()}" if gender == 'Rooster'
    else f"{fake.first_name_female()} {fake.last_name()}"
    for gender in chicken_df['gender']
]

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name
0,25804801,1,Rooster,NaN,Raymond Hansen
1,37570562,1,Rooster,NaN,Gary Stokes
2,46751747,1,Rooster,NaN,Ryan Joyce
3,29038594,1,Rooster,NaN,James Blair
4,43585541,1,Rooster,NaN,Todd Glenn
...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson
996,17602553,3,Hen,17.0,Valerie Castro
997,95074299,3,Hen,30.0,Isabel Gregory
998,80680956,3,Hen,20.0,Sophia Dillon


In [85]:
# generate random feather colors for each chicken
def random_feather_color():
    colors = ['white', 'black', 'brown', 'red', 'gray', 'gold'] # ideally these are realistic colors
    return random.choice(colors)

chicken_df['feather_color'] = [random_feather_color() for i in range(len(chicken_df))]

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color
0,25804801,1,Rooster,NaN,Raymond Hansen,red
1,37570562,1,Rooster,NaN,Gary Stokes,brown
2,46751747,1,Rooster,NaN,Ryan Joyce,white
3,29038594,1,Rooster,NaN,James Blair,white
4,43585541,1,Rooster,NaN,Todd Glenn,black
...,...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson,brown
996,17602553,3,Hen,17.0,Valerie Castro,gold
997,95074299,3,Hen,30.0,Isabel Gregory,gold
998,80680956,3,Hen,20.0,Sophia Dillon,black


In [86]:
# this is commented out to avoid running it again, but it's here for reference

# using a free API to get the billboard top 10 songs from a given date
# import requests

# url = "https://billboard-api2.p.rapidapi.com/hot-100"

# querystring = {"date":"2023-03-01","range":"1-10"}

# headers = {
#     # i know this is bad practice, but this is a free API key capped at 30 reqs/month
# 	"X-RapidAPI-Key": "79daae12cdmsh6e4351700c61e60p18544ejsnb1b1f2f8b40d",
# 	"X-RapidAPI-Host": "billboard-api2.p.rapidapi.com"
# }

# response = requests.request("GET", url, headers=headers, params=querystring)

top_10 = response.json()

def create_song_list(json_data):
    content = json_data['content']
    song_list = [f"{song['title']} by {song['artist']}" for song in content.values()]
    return song_list

song_list = create_song_list(top_10)

chicken_df['favorite_song'] = [random.choice(song_list) for i in range(len(chicken_df))]

chicken_df


,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song
0,25804801,1,Rooster,NaN,Raymond Hansen,red,Flowers by Miley Cyrus
1,37570562,1,Rooster,NaN,Gary Stokes,brown,Anti-Hero by Taylor Swift
2,46751747,1,Rooster,NaN,Ryan Joyce,white,I'm Good (Blue) by David Guetta & Bebe Rexha
3,29038594,1,Rooster,NaN,James Blair,white,Cuff It by Beyonce
4,43585541,1,Rooster,NaN,Todd Glenn,black,Kill Bill by SZA
...,...,...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson,brown,Anti-Hero by Taylor Swift
996,17602553,3,Hen,17.0,Valerie Castro,gold,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice"
997,95074299,3,Hen,30.0,Isabel Gregory,gold,Kill Bill by SZA
998,80680956,3,Hen,20.0,Sophia Dillon,black,Kill Bill by SZA


In [87]:
# function to get parent hens and roosters for each generation
def get_parents(chicken_df, current_generation):
    parents_df = chicken_df[chicken_df['generation'] == current_generation - 1]
    hens = parents_df[parents_df['gender'] == 'Hen']['egg_id'].tolist()
    roosters = parents_df[parents_df['gender'] == 'Rooster']['egg_id'].tolist()
    return hens, roosters

# function to assign parent IDs to each chicken
def assign_parents(chicken_df, generation, hens, roosters):
    chicken_df.loc[chicken_df['generation'] == generation, 'parent_hen_id'] = [
        random.choice(hens) for i in range(len(chicken_df[chicken_df['generation'] == generation]))]
    chicken_df.loc[chicken_df['generation'] == generation, 'parent_rooster_id'] = [
        random.choice(roosters) for i in range(len(chicken_df[chicken_df['generation'] == generation]))]

# get parent hens and roosters for each generation
gen2_hen_parents, gen2_rooster_parents = get_parents(chicken_df, 2)
gen3_hen_parents, gen3_rooster_parents = get_parents(chicken_df, 3)

# assign parent IDs for chickens in generation 2 and 3
assign_parents(chicken_df, 2, gen2_hen_parents, gen2_rooster_parents)
assign_parents(chicken_df, 3, gen3_hen_parents, gen3_rooster_parents)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id
0,25804801,1,Rooster,NaN,Raymond Hansen,red,Flowers by Miley Cyrus,NaN,NaN
1,37570562,1,Rooster,NaN,Gary Stokes,brown,Anti-Hero by Taylor Swift,NaN,NaN
2,46751747,1,Rooster,NaN,Ryan Joyce,white,I'm Good (Blue) by David Guetta & Bebe Rexha,NaN,NaN
3,29038594,1,Rooster,NaN,James Blair,white,Cuff It by Beyonce,NaN,NaN
4,43585541,1,Rooster,NaN,Todd Glenn,black,Kill Bill by SZA,NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson,brown,Anti-Hero by Taylor Swift,96727910.0,21805472.0
996,17602553,3,Hen,17.0,Valerie Castro,gold,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",49883805.0,21451199.0
997,95074299,3,Hen,30.0,Isabel Gregory,gold,Kill Bill by SZA,98300772.0,40325333.0
998,80680956,3,Hen,20.0,Sophia Dillon,black,Kill Bill by SZA,75719244.0,55978392.0


In [88]:
# long function to find first cousins for a given chicken
def find_cousins(chicken_id, df):
    chicken_df = df.loc[df['egg_id'] == chicken_id]
    if chicken_df.empty:
        return []

    chicken_df = chicken_df.iloc[0]
    parent_hen = chicken_df['parent_hen_id']
    parent_rooster = chicken_df['parent_rooster_id']

    parent_hen_df = df.loc[df['egg_id'] == parent_hen]
    if parent_hen_df.empty:
        return []

    parent_hen_df = parent_hen_df.iloc[0]
    parent_hen_parent_hen = parent_hen_df['parent_hen_id']  # grandmother hen side
    parent_hen_parent_rooster = parent_hen_df['parent_rooster_id']  # grandfather hen side

    parent_hen_siblings = df.loc[
        (df['parent_hen_id'] == parent_hen_parent_hen)
        & (df['parent_rooster_id'] == parent_hen_parent_rooster)
        & (df['egg_id'] != parent_hen)
    ]

    cousins_parent_hen_side = df.loc[
        (df['parent_hen_id'].isin(parent_hen_siblings['egg_id']))
        | (df['parent_rooster_id'].isin(parent_hen_siblings['egg_id']))
    ]

    parent_rooster_df = df.loc[df['egg_id'] == parent_rooster]
    if parent_rooster_df.empty:
        return []

    parent_rooster_df = parent_rooster_df.iloc[0]
    parent_rooster_parent_hen = parent_rooster_df['parent_hen_id']  # grandmother rooster side
    parent_rooster_parent_rooster = parent_rooster_df['parent_rooster_id']  # grandfather rooster side

    parent_rooster_siblings = df.loc[
        (df['parent_hen_id'] == parent_rooster_parent_hen)
        & (df['parent_rooster_id'] == parent_rooster_parent_rooster)
        & (df['egg_id'] != parent_rooster)
    ]

    cousins_parent_rooster_side = df.loc[
        (df['parent_hen_id'].isin(parent_rooster_siblings['egg_id']))
        | (df['parent_rooster_id'].isin(parent_rooster_siblings['egg_id']))
    ]

    all_cousins = pd.concat([cousins_parent_rooster_side, cousins_parent_hen_side]).drop_duplicates()['egg_id'].to_list()

    return all_cousins

# only finding cousins for chicken in gen 3, since we don't know who grandparents are of chickens in gen 2
chicken_df['cousins'] = chicken_df.apply(lambda row: find_cousins(row['egg_id'], chicken_df) if row['generation'] == 3 else [], axis=1)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id,cousins
0,25804801,1,Rooster,NaN,Raymond Hansen,red,Flowers by Miley Cyrus,NaN,NaN,[]
1,37570562,1,Rooster,NaN,Gary Stokes,brown,Anti-Hero by Taylor Swift,NaN,NaN,[]
2,46751747,1,Rooster,NaN,Ryan Joyce,white,I'm Good (Blue) by David Guetta & Bebe Rexha,NaN,NaN,[]
3,29038594,1,Rooster,NaN,James Blair,white,Cuff It by Beyonce,NaN,NaN,[]
4,43585541,1,Rooster,NaN,Todd Glenn,black,Kill Bill by SZA,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson,brown,Anti-Hero by Taylor Swift,96727910.0,21805472.0,"[86187229, 56067902, 21880733]"
996,17602553,3,Hen,17.0,Valerie Castro,gold,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",49883805.0,21451199.0,[]
997,95074299,3,Hen,30.0,Isabel Gregory,gold,Kill Bill by SZA,98300772.0,40325333.0,[]
998,80680956,3,Hen,20.0,Sophia Dillon,black,Kill Bill by SZA,75719244.0,55978392.0,[]


In [89]:
# creating a 'random cousin' column for use in name tags

chicken_df['random_cousin'] = chicken_df['cousins'].apply(lambda x: random.choice(x) if x else None)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id,cousins,random_cousin
0,25804801,1,Rooster,NaN,Raymond Hansen,red,Flowers by Miley Cyrus,NaN,NaN,[],NaN
1,37570562,1,Rooster,NaN,Gary Stokes,brown,Anti-Hero by Taylor Swift,NaN,NaN,[],NaN
2,46751747,1,Rooster,NaN,Ryan Joyce,white,I'm Good (Blue) by David Guetta & Bebe Rexha,NaN,NaN,[],NaN
3,29038594,1,Rooster,NaN,James Blair,white,Cuff It by Beyonce,NaN,NaN,[],NaN
4,43585541,1,Rooster,NaN,Todd Glenn,black,Kill Bill by SZA,NaN,NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,72511480,3,Hen,24.0,Melanie Nelson,brown,Anti-Hero by Taylor Swift,96727910.0,21805472.0,"[86187229, 56067902, 21880733]",56067902.0
996,17602553,3,Hen,17.0,Valerie Castro,gold,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",49883805.0,21451199.0,[],NaN
997,95074299,3,Hen,30.0,Isabel Gregory,gold,Kill Bill by SZA,98300772.0,40325333.0,[],NaN
998,80680956,3,Hen,20.0,Sophia Dillon,black,Kill Bill by SZA,75719244.0,55978392.0,[],NaN


# 2b. Questions about the synthetic data


##### What can we do to make these records as realistic as possible?
We can do a number of things:
- Include timestamps for when each chicken was born and make sure that the children of a hen are birthed at realistic rates and timeframes
- Have a realistic distribution of hens and roosters
- Make sure that mating patterns reflect realistic chicken behavior
- Account for the fact that some eggs may be invalid / some hens may be infertile
- Make sure that we've accounted for various types of roosters/hens having certain sets of possible colors

Assumptions I made to attempt to make the data realistic in a timeboxed manner:
- The farm imported 100 hens and roosters to kickstart their farm
- Each generation thereafter grew at an increasing rate
- The farmers elected to keep a 50-50 split by gender within each generation after generation 1
- Exactly one third of the incubation locations are by windows, but I didn't apply this condition strictly to egg births to account for the fact that some eggs may break/die etc.
- Allowed for a hen to mate with multiple roosters

##### How could a government official check whether the data is faked or not?
- It may be suspicious that there is an exact 50-50 gender split within each generation, especially because hens can mate with multiple roosters so a 50-50 split may not be necessary to have a healthy farm
- If I had included birthing timestamps, a government official could check to make sure they follow known egg-laying rates for hens
- A government official could request access to our database and check when these records were first created

##### What can we do to cover up these checks?
- For data quality, we can spend time getting a deep understanding for what real chicken farm data looks like and make our data meet it exactly. (Things like lifespans, distributions of genders, distributions of the number of eggs per hen, mating schedules, mating partner counts, etc.)
- We can fake / overwrite created_at timestamps in our database to make it seem like the data is reflective of when chickens were birthed

##### What can a government official check to see whether or not we're covering up their checks?
- Checking database logs in e.g. postgres


##### Links to things I looked up:
- https://www.mypetchicken.com/blogs/faqs/can-different-chicken-breeds-cross-breed
- https://www.thehappychickencoop.com/how-do-chickens-mate/
- https://starmilling.com/chicken-feather-colors-101/

## 3. Generate the name tags table

In [90]:
name_tags = duckdb.query("""
        select
            base.name,
            base.favorite_song,
            base.generation,
            p_hen.name as parent_hen_name,
            p_hen.incubation_hall_location_id as parent_hen_location,
            p_roo.name as parent_rooster_name,
            p_roo.incubation_hall_location_id as parent_rooster_location,
            gp_hen_hen_side.name as maternal_grandma_name,
            gp_hen_hen_side.incubation_hall_location_id as maternal_grandma_location,
            gp_roo_hen_side.name as maternal_grandpa_name,
            gp_roo_hen_side.incubation_hall_location_id as maternal_grandpa_location,
            gp_hen_roo_side.name as paternal_grandma_name,
            gp_hen_roo_side.incubation_hall_location_id as paternal_grandma_location,       
            gp_roo_roo_side.name as paternal_grandpa_name,
            gp_roo_roo_side.incubation_hall_location_id as paternal_grandpa_location,
            cousins.name as cousin
        from chicken_df base
        left join chicken_df p_hen
            on base.parent_hen_id = p_hen.egg_id
        left join chicken_df p_roo
            on base.parent_rooster_id = p_roo.egg_id
        left join chicken_df gp_hen_hen_side
            on p_hen.parent_hen_id = gp_hen_hen_side.egg_id    
        left join chicken_df gp_roo_hen_side
            on p_hen.parent_rooster_id = gp_roo_hen_side.egg_id        
        left join chicken_df gp_roo_roo_side
            on p_roo.parent_rooster_id = gp_roo_roo_side.egg_id  
        left join chicken_df gp_hen_roo_side
            on p_roo.parent_hen_id = gp_hen_roo_side.egg_id 
        left join chicken_df cousins
            on base.random_cousin = cousins.egg_id             
""").to_df()

name_tags

,name,favorite_song,generation,parent_hen_name,parent_hen_location,parent_rooster_name,parent_rooster_location,maternal_grandma_name,maternal_grandma_location,maternal_grandpa_name,maternal_grandpa_location,paternal_grandma_name,paternal_grandma_location,paternal_grandpa_name,paternal_grandpa_location,cousin
0,Thomas Morgan,Flowers by Miley Cyrus,2,Morgan English,NaN,John Gutierrez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kristopher Chapman,Die For You by The Weeknd,2,Veronica Walker,NaN,Adam Lewis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jason Flores,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",2,Sarah Barnett,NaN,Joshua Baker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ronald Rowe,Last Night by Morgan Wallen,2,Dana Schwartz,NaN,Tyler Hutchinson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,William Carrillo,I'm Good (Blue) by David Guetta & Bebe Rexha,2,Samantha Roberts,NaN,Gregory Martin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Dana Meyer,Last Night by Morgan Wallen,3,Jennifer Stout,8.0,Harry Hicks,14.0,Rebecca Brown,NaN,Charles Powell,NaN,Cynthia Perry,NaN,Raymond Hansen,NaN,Justin Meyers
996,Melissa Martinez,Kill Bill by SZA,3,Melissa Chandler,23.0,Jacob Wolfe,22.0,Samantha Roberts,NaN,Willie Reynolds,NaN,Lisa Campbell,NaN,Charles Powell,NaN,Brian Robertson
997,Adam Mann,I'm Good (Blue) by David Guetta & Bebe Rexha,3,Veronica Oneill,3.0,Matthew Edwards,12.0,Sarah Barnett,NaN,Kyle Rojas,NaN,Cynthia Perry,NaN,Raymond Hansen,NaN,Nina Duran
998,Johnny Moore,I'm Good (Blue) by David Guetta & Bebe Rexha,3,Kelly Alexander,14.0,Harry Hicks,14.0,Morgan English,NaN,Scott Barr,NaN,Cynthia Perry,NaN,Raymond Hansen,NaN,Adam Mann


#### Load dataframes to bigquery for metabase

In [91]:
import os
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/shivensamant/Downloads/just-vent-377014-518fa303dc42.json'

bigquery_client = bigquery.Client()

In [92]:
import pandas_gbq, json

# load chicken-df table
project_id = "just-vent-377014"
dataset_id = "scratch"
table_id = "chicken-df"

destination_table = f"{project_id}.{dataset_id}.{table_id}"

# convert list of cousins to json string for storage in bigquery
chicken_df['cousins'] = chicken_df['cousins'].apply(json.dumps)


pandas_gbq.to_gbq(chicken_df, destination_table, project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 21845.33it/s]


In [93]:
# load incubation_hall table
project_id = "just-vent-377014"
dataset_id = "scratch"
table_id = "incubation_hall"

destination_table = f"{project_id}.{dataset_id}.{table_id}"

pandas_gbq.to_gbq(incubation_hall, destination_table, project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 10010.27it/s]
